# [Digital image steganography: Survey and analysis of current methods](http://www.sciencedirect.com/science/article/pii/S0165168409003648)

## DCT
* Li and Wang modified the quantization table itself, and then inserted hidden bits in the middle frequency coefficients. This gave them 36 coefficients to embed in per block, and thus a good payload. [39]
![Modified QT](QT.png)
* JSteg is easily detected with the chi squared test. [3]
* OutGuess used a pRNG to select DCT coefficients, which thwarts the chi squared test. However, an extended version of the chi squared test may work. [3]
* F5 embeds only into non-zero AC DCT coeefficients by decreasing the absolute value of the coefficient by 1. The same bit has to be re-embedded in case the original is 1 or -1. Fridrich et al. came up with a detection method for it. [33]
* Perturbed Quantization has minimal distortion but low capacity. [51]

## Adaptive
Analyzes statistical global features of the cover image, and tries to embed in noisy areas.
* MB1 is model based [58]. Can be broken using first-order statistics [63]. Can also be detected through difference in "blockiness" [64]. Enhanced version to avoid blockiness is MB2 [58]. MB2 can still be detected.
* Chin-Chen et al. estimate the degree of smoothness between pixels, with high capacity. [65]
* A Block Complexity based Data Embedding replaces suitable noisy blocks of pixels with noisy blocks containing data. [66]
* Genetic algorithm breaks detection by artificially counterfeiting statistical features. Time complexity may be an issue. [74]
* Genetic algorithm that uses +/-1 technique on JPEG. [64]
* Kong et al. calculates entropy for each region, which dictates embedding strength of LSB. [76]

## Other 2D image
* Discrete wavelet transform [55] [57] [67]
* Nakamura and Zhao morphing process [79]
* Zeki and Azizah intermediate significant bit algorithm [80]

* New detection methods utilize neural networks [46] [56]

## 3D graphics
* Chao et al. [77]
* Bogomjakov et al. [78]

## Video
* McKeon used 2D DFT for steganography in movies. [44]

## List of tools
![List of tools](Tools.PNG)

# [Hide and Seek: An Introduction to Steganography](http://citi.umich.edu/u/provos/papers/practical.pdf)
# JSteg and Outguess

## JSteg
JSteg sequentially replaces the LSB of DCT coefficients. It is detected by the chi-squared test because the embedding reduces the frequency difference between adjacent DCT coefficient values.
![JSteg algorithm](JSteg.png)

## Chi-squared attack
We do not know the cover image, but we know the sum of adjacent DCT coefficients remain invariant (since LSB only switches values with their adjacent pair). This lets us compute an expected distribution that would be generated by a stego image, and compare that to the actual distribution of the image being analyzed, using the chi-squared statistical test. The chi-squared value gives a probability that the analyzed image is a stego image.
![Chi squared test](ChiSquared.png)
The attack can take a sample of the DCT coefficients, so that it can show exactly which part of the image has been embedded into (and thus how long the secret message is, at least against JSteg). The samples start at the beginning of the image, and each measurement increases the sample size, until the sample size is the entire image.

## Outguess
Outguess is an improvement on JSteg that uses a pRNG to select DCT coefficients for embedding at random. It is detected by an extended version of the chi-squared test that is more sensitive to local distortions.
![Outguess algorithm](Outguess.png)
An improved version of Outguess uses the unused LSBs of the cover image by manipulating them to get rid of statistical anomalies and thus counter the extended chi-squared attack. However, this improved version can be detected by two other methods: a support vector machine trained on stego and non-stego images by Lyu et al. or a special method designed against Outguess that also determines hidden message length by Fridrich et al.

## Extended chi-squared attack
Instead of increasing the sample size with constant position, the extended version uses constant sample size but slides the position across the image's entire range. With a given sample size, we increase the sample position by 1 percent for every chi-squared calculation. Then we take the sum of probability of embedding for all samples, and then compare it to a chosen detection threshold.

To choose a sample size, we select an expected distribution that should cause a negative test result. Taking the arithmetic mean of two unrelated coefficients (2i-1 and 2i), we can use a binary search to find a sample size that does not show a correlation between the actual distribution and the expected distribution from unrelated coefficients.

## F5
F5 embeds data by decrementing the absolute value of DCT coefficients. Since it does not create fixed value pairs, it cannot be detected by the chi-squared test. F5 uses a pRNG with a shared secret seed to permutate the order of embedding across DCT coefficients (like Outguess does).

F5 uses the Hamming code to encode a k-bit message word into a 2<sup>k-1</sup>-bit code word. The message block size k is calculated from the message length and the number of non-zero non-DC coefficients. Thus, each DCT coefficient is used to embed 1 bit of a message word (via its LSB, which decrementing changes). For each k-bit message word, the Hamming code requires changing at most 1 bit of the existing code word in the cover image to match. If a coefficient is decremented to 0, that coefficient is discarded from the embedding group (the coefficient is kept at 0 so that it is ignored during decoding). Then, the next nonzero AC coefficient is added to the embedding group, and so on until embedding is successful without decrementing a coefficient to 0.
![F5 algorithm](F5.png)
Fridrich et al. discovered an attack that detects how F5 changes the DCT coefficients.

## Fridrich's Attack
Let h<sub>uv</sub>(d) be the number of AC coefficients at frequency/position (u,v) in its DCT block with value d in the cover image. Let H<sub>uv</sub>(d) be the corresponding function for the stego image. Then the change rate beta of AC coefficients can be estimated via the least square approximation.
![Beta approximation](Beta.png)
h<sub>uv</sub>(d) is estimated by cropping the stego image by 4 pixel columns and then recompressing to DCT.